In [1]:
import pandas as pd
from pathlib import Path
import numpy as np
import os
import astroalign as aa

In [2]:
#filter specific file names
luminance = 'star_Light_Luminance_5_secs_001_reduced_stars.csv'
red = 'star_Light_Red_30_secs_001_reduced_stars.csv'
green = 'star_Light_Green_30_secs_001_reduced_stars.csv'
blue = 'star_Light_Blue_30_secs_001_reduced_stars.csv'

#set current filter
ref_csv_name = luminance
current_filter = 'Luminance'

read_path = Path(f'./RZ Cep/All_RZ_Cep/2022_12_07/RZ_Cep/CSVs/Unmatched Stars/{current_filter}')
file_list = [f for f in os.listdir(read_path)if f.endswith('_stars.csv') and 
             f != ref_csv_name]
write_path = Path(f'./RZ Cep/All_RZ_Cep/2022_12_07/RZ_Cep/CSVs/Matched Stars/{current_filter}')

In [3]:
#transform star lists
#set up arrays to hold transforms, source lists, target lists, number of matching stars and all the bad files
transfs = []
source_lists = [] 
target_lists = []
number_matching_stars = []
bad_files = []

#read in the target/reference dataframe
df_target = pd.read_csv(read_path / ref_csv_name, delimiter=',')
#isolate star positions
target = pd.concat([df_target['x_center'], df_target['y_center']], axis=1)


for i in range(len(file_list)):
        #print(i,len(file_list))
        #print(file_list[i])
        #read in source dataframe which we match to the target
        df_source = pd.read_csv(read_path / file_list[i], index_col=0)
        #isolate star positions
        source = pd.concat([df_source['x_center'],df_source['y_center']], axis=1)
        #find the transform for the current source image, which we do not apply
        #when max control points is set to 75+, falsely duplicates some stars
        transf, (source_list, target_list) = aa.find_transform(source,target,max_control_points=50)
        #array appending
        transfs.append(transf)
        source_lists.append(source_list)
        target_lists.append(target_list)
        number_matching_stars.append(len(target_list))

        #match source_list indexes to indexes in source dataframe
        df_matched_source = df_source.iloc[pd.Index(df_source['x_center']).get_indexer(source_list[:,0])]
        #reset indexes so they are the same
        df_matched_source.reset_index(inplace=True)

        #match target_list indexes to indexes in target dataframe
        df_matched_target = df_target.iloc[pd.Index(df_target['x_center']).get_indexer(target_list[:,0])]
        #reset indexes so they are the same
        df_matched_target.reset_index(inplace=True)

        #join matched star ids to the course dataframe to create a new dataframe
        df_out = df_matched_source.join(df_matched_target['star_id'], rsuffix='_ref')

        #update file name and write output matched dataframe to a new csv
        file_name = file_list[i].replace('_stars.csv','_matched.csv')
        print(file_name)
        df_out.to_csv(write_path / file_name)

#rename target csv and write to same location as matched csvs
ref_csv_name = ref_csv_name.replace('_stars', '_matched')
df_target.to_csv(write_path / ref_csv_name)
#print any bad file names for closer inspection
print(bad_files)

C:\ProgramData\Anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


star_Light_Luminance_5_secs_002_reduced_matched.csv
star_Light_Luminance_5_secs_003_reduced_matched.csv
star_Light_Luminance_5_secs_004_reduced_matched.csv
star_Light_Luminance_5_secs_005_reduced_matched.csv
star_Light_Luminance_5_secs_006_reduced_matched.csv
star_Light_Luminance_5_secs_007_reduced_matched.csv
star_Light_Luminance_5_secs_008_reduced_matched.csv
star_Light_Luminance_5_secs_009_reduced_matched.csv
star_Light_Luminance_5_secs_010_reduced_matched.csv
star_Light_Luminance_5_secs_011_reduced_matched.csv
star_Light_Luminance_5_secs_012_reduced_matched.csv
star_Light_Luminance_5_secs_013_reduced_matched.csv
star_Light_Luminance_5_secs_014_reduced_matched.csv
star_Light_Luminance_5_secs_015_reduced_matched.csv
star_Light_Luminance_5_secs_016_reduced_matched.csv
star_Light_Luminance_5_secs_017_reduced_matched.csv
star_Light_Luminance_5_secs_018_reduced_matched.csv
star_Light_Luminance_5_secs_019_reduced_matched.csv
star_Light_Luminance_5_secs_020_reduced_matched.csv
star_Light_L

star_Light_Luminance_5_secs_160_reduced_matched.csv
star_Light_Luminance_5_secs_161_reduced_matched.csv
star_Light_Luminance_5_secs_162_reduced_matched.csv
star_Light_Luminance_5_secs_163_reduced_matched.csv
star_Light_Luminance_5_secs_164_reduced_matched.csv
star_Light_Luminance_5_secs_165_reduced_matched.csv
star_Light_Luminance_5_secs_166_reduced_matched.csv
star_Light_Luminance_5_secs_167_reduced_matched.csv
star_Light_Luminance_5_secs_168_reduced_matched.csv
star_Light_Luminance_5_secs_169_reduced_matched.csv
star_Light_Luminance_5_secs_170_reduced_matched.csv
star_Light_Luminance_5_secs_171_reduced_matched.csv
star_Light_Luminance_5_secs_172_reduced_matched.csv
[]


In [4]:
df_out

,index,x_center,y_center,annulus_bkg,counts,sky_corrected_counts,counts_error,flux,flux_error,instrumental_mag,mag_error,saturated?,FWHM,date_time,exp_time,sharpness,roundness_sym,roundness_gauss,ellipticity,star_id
0,25,209.239507,2825.264891,2.505184e+06,2.575666e+06,70482.280926,265.484992,70482.280926,265.484992,-12.120200,-0.004090,False,15.051466,2022-12-08T04:13:12.394,5.0,0.890281,0.040791,-0.032355,0.001846,TYC 4273-900-1
1,20,4293.251805,2057.021906,2.287386e+06,2.370623e+06,83236.387553,288.507171,83236.387553,288.507171,-12.300783,-0.003763,False,15.051466,2022-12-08T04:13:12.394,5.0,0.735560,-0.082252,0.474965,0.002578,Gaia DR3 2212402667093547008
2,29,1616.984371,1658.175344,2.394239e+06,2.446679e+06,52440.075489,228.997981,52440.075489,228.997981,-11.799158,-0.004741,False,15.051466,2022-12-08T04:13:12.394,5.0,0.326227,0.000093,0.764096,0.003658,UCAC4 774-052557
3,5,271.958600,1400.408613,2.473716e+06,2.678241e+06,204525.501729,452.244958,204525.501729,452.244958,-13.276869,-0.002401,False,15.051466,2022-12-08T04:13:12.394,5.0,0.613925,0.006921,0.805636,0.004948,TYC 4273-876-1
4,28,2038.643646,1039.603358,2.342624e+06,2.407549e+06,64925.047345,254.803939,64925.047345,254.803939,-12.031031,-0.004261,False,15.051466,2022-12-08T04:13:12.394,5.0,0.371164,0.315533,0.623129,0.003073,TYC 4273-822-1
5,7,1290.435616,1397.832667,2.386602e+06,2.593070e+06,206468.021566,454.387524,206468.021566,454.387524,-13.287132,-0.002389,False,15.051466,2022-12-08T04:13:12.394,5.0,0.797094,-0.013141,0.839828,0.003155,IRAS 22404+6450
6,4,4083.391285,2669.526605,2.302752e+06,2.605126e+06,302373.413614,549.884909,302373.413614,549.884909,-13.701359,-0.001974,False,15.051466,2022-12-08T04:13:12.394,5.0,0.664638,0.112172,0.633957,0.007165,UCAC4 774-052682
7,3,1565.256754,2135.602484,2.415975e+06,2.665322e+06,249347.403401,499.346977,249347.403401,499.346977,-13.492012,-0.002174,False,15.051466,2022-12-08T04:13:12.394,5.0,0.403484,0.173411,0.652024,0.001416,BD+64 1700
8,8,3898.645467,1907.112484,2.320073e+06,2.468578e+06,148504.722658,385.363105,148504.722658,385.363105,-12.929351,-0.002817,False,15.051466,2022-12-08T04:13:12.394,5.0,0.921453,0.023369,0.936540,0.004284,TYC 4273-668-1
9,1,314.622313,3001.625930,2.515719e+06,3.182086e+06,666366.344738,816.312651,666366.344738,816.312651,-14.559283,-0.001330,False,15.051466,2022-12-08T04:13:12.394,5.0,0.667411,-0.195410,0.658553,0.009865,TYC 4273-1351-1


In [5]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(df_out)

    index   id      xcenter      ycenter  aperture_sum  Circ_sum_flux  \
0      26   38  4170.190663  2326.346609  1.166665e+06   1.166665e+06   
1      20   47  2954.543573  1261.335680  1.214089e+06   1.214089e+06   
2      46   37  2303.585121  1829.315507  1.126316e+06   1.126316e+06   
3      33   29  2542.947596  1160.339989  1.139553e+06   1.139553e+06   
4      34   32   895.022587   107.817852  1.144276e+06   1.144276e+06   
5      31   46  1296.605336  3108.442934  1.158606e+06   1.158606e+06   
6       2    3  3224.402872  1059.051908  2.080708e+06   2.080708e+06   
7      17   80  2961.508691  1252.125554  1.221280e+06   1.221280e+06   
8      36   25  1404.190303   239.918934  1.139284e+06   1.139284e+06   
9      11   11  1295.526749  1555.366730  1.434449e+06   1.434449e+06   
10     13   22  4289.197786  2269.846998  1.254183e+06   1.254183e+06   
11     32   21   836.636876   598.697787  1.159151e+06   1.159151e+06   
12     30   33  2200.744475   954.908543  1.140130e

In [9]:
df_matched_target

,index,Unnamed: 0,id,xcenter,ycenter,aperture_sum,Circ_sum_flux,Circ_area,Max_Flux,FWHM_form_aperature,...,Air mass,Azimuth,Altitude,Date Observation,Exposure time /s,Filter,File,ra_from_astrometry_deg,dec_from_astrometry_deg,star_id
0,20,20,42,2936.024056,1284.775663,1.721996e+06,1.721996e+06,6034.979809,1351.947362,12.522612,...,1.08652,320.1292,66.96391,2022-12-07T20:08:35.241,5.0,Luminance,star_Light_Luminance_5_secs_001_reduced.fits,339.613410,64.797298,TYC 4273-659-1
1,26,26,20,2584.506950,424.719094,1.684754e+06,1.684754e+06,6034.979809,2324.415622,12.522612,...,1.08652,320.1292,66.96391,2022-12-07T20:08:35.241,5.0,Luminance,star_Light_Luminance_5_secs_001_reduced.fits,339.366868,64.865855,Gaia DR2 2211583698432680064
2,47,47,49,167.081670,2985.539215,1.631396e+06,1.631396e+06,6034.979809,1211.232503,12.522612,...,1.08652,320.1292,66.96391,2022-12-07T20:08:35.241,5.0,Luminance,star_Light_Luminance_5_secs_001_reduced.fits,340.318352,65.118201,TYC 4273-1186-1
3,41,41,31,2031.676481,1232.713643,1.622897e+06,1.622897e+06,6034.979809,1556.987399,12.522612,...,1.08652,320.1292,66.96391,2022-12-07T20:08:35.241,5.0,Luminance,star_Light_Luminance_5_secs_001_reduced.fits,339.652197,64.918304,TYC 4273-1022-1
4,2,2,2,3205.683885,1082.783297,2.632768e+06,2.632768e+06,6034.979809,11148.792813,12.522612,...,1.08652,320.1292,66.96391,2022-12-07T20:08:35.241,5.0,Luminance,star_Light_Luminance_5_secs_001_reduced.fits,339.534389,64.766822,TYC 4273-1351-1
5,37,37,28,1386.161966,262.718210,1.642111e+06,1.642111e+06,6034.979809,1660.473821,12.522612,...,1.08652,320.1292,66.96391,2022-12-07T20:08:35.241,5.0,Luminance,star_Light_Luminance_5_secs_001_reduced.fits,339.387867,65.028629,TYC 4273-149-1
6,25,25,18,2323.285182,202.005926,1.682397e+06,1.682397e+06,6034.979809,2153.678670,12.522612,...,1.08652,320.1292,66.96391,2022-12-07T20:08:35.241,5.0,Luminance,star_Light_Luminance_5_secs_001_reduced.fits,339.312942,64.906081,TYC 4273-293-1
7,17,17,16,2387.056199,2243.158504,1.748753e+06,1.748753e+06,6034.979809,2375.688980,12.522612,...,1.08652,320.1292,66.96391,2022-12-07T20:08:35.241,5.0,Luminance,star_Light_Luminance_5_secs_001_reduced.fits,339.945185,64.844807,GSH 109+06-32
8,15,15,19,1748.893272,3029.988309,1.767811e+06,1.767811e+06,6034.979809,2436.338387,12.522612,...,1.08652,320.1292,66.96391,2022-12-07T20:08:35.241,5.0,Luminance,star_Light_Luminance_5_secs_001_reduced.fits,340.230523,64.908059,TYC 4273-922-1
9,14,14,21,4270.201924,2293.801691,1.763994e+06,1.763994e+06,6034.979809,2302.759107,12.522612,...,1.08652,320.1292,66.96391,2022-12-07T20:08:35.241,5.0,Luminance,star_Light_Luminance_5_secs_001_reduced.fits,339.844542,64.594482,1RXS J223959.9+643330


In [6]:
transfs

[<SimilarityTransform(matrix=
     [[ 9.99953821e-01, -4.96092034e-04, -1.79494103e+01],
      [ 4.96092034e-04,  9.99953821e-01,  2.21778042e+01],
      [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00]]) at 0x22b55185a90>,
 <SimilarityTransform(matrix=
     [[ 1.00004210e+00, -6.01676467e-04, -1.85905097e+01],
      [ 6.01676467e-04,  1.00004210e+00,  2.19828318e+01],
      [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00]]) at 0x22b54d6ebd0>,
 <SimilarityTransform(matrix=
     [[ 9.99993851e-01, -6.93912934e-04, -2.10117567e+01],
      [ 6.93912934e-04,  9.99993851e-01,  2.22862102e+01],
      [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00]]) at 0x22b5622bfd0>,
 <SimilarityTransform(matrix=
     [[ 9.99964135e-01, -7.77140131e-04, -1.81365646e+01],
      [ 7.77140131e-04,  9.99964135e-01,  2.41147926e+01],
      [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00]]) at 0x22b5622bb50>,
 <SimilarityTransform(matrix=
     [[ 9.99998898e-01, -9.22001331e-04, -2.09998652e+01],
   